In [ ]:
!pip install tensorflow

In [ ]:
!pip install PIL

In [ ]:
!pip install scipy

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


In [7]:
# Parameter
IMG_SIZE = (224,224)  # Ukuran gambar untuk MobileNetV2
BATCH_SIZE = 32
EPOCHS = 35
LEARNING_RATE = 0.001
FINE_TUNE_EPOCHS = 20  # Tambahan epoch untuk fine-tuning
FINE_TUNE_LR = 1e-5    # Learning rate untuk fine-tuning

# Direktori dataset
train_dir = "train_batik"
test_dir = "test_batik_2"

# Augmentasi Data
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

test_datagen = ImageDataGenerator(rescale=1.0/255)


# Generator Data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

# Informasi jumlah kelas
num_classes = len(train_generator.class_indices)

Found 928 images belonging to 29 classes.


Found 232 images belonging to 29 classes.


In [8]:
# Model Transfer Learning
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Bekukan layer pada model dasar untuk transfer learning

# Tambahkan lapisan khusus
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Dropout untuk mencegah overfitting
x = Dense(256, activation="relu")(x)
predictions = Dense(num_classes, activation="softmax")(x)

# Buat model
model = Model(inputs=base_model.input, outputs=predictions)

# Kompilasi model untuk transfer learning
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Callback
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Pelatihan awal (transfer learning)
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator,
    callbacks=[reduce_lr, early_stopping]
)

Epoch 1/35
29/29 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.0590 - loss: 3.7042 - val_accuracy: 0.2845 - val_loss: 2.6953 - learning_rate: 0.0010
Epoch 2/35
29/29 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.2982 - loss: 2.5821 - val_accuracy: 0.4267 - val_loss: 2.1454 - learning_rate: 0.0010
Epoch 3/35
29/29 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.4379 - loss: 1.9871 - val_accuracy: 0.4526 - val_loss: 1.8724 - learning_rate: 0.0010
Epoch 4/35
29/29 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.5125 - loss: 1.6049 - val_accuracy: 0.5302 - val_loss: 1.6529 - learning_rate: 0.0010
Epoch 5/35
29/29 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.5875 - loss: 1.3623 - val_accuracy: 0.5603 - val_loss: 1.5526 - learning_rate: 0.0010
Epoch 6/35
29/29 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.6032 - loss: 1.2228 - val_accuracy: 0.5302 - val_loss: 1.5856 - learning_rate: 0.0010
Epoch 7/35
29/29 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.6674 - loss: 1.0990 - val_accuracy: 

In [9]:
model.save("mobilenet_batik_oldcode.h5") # Menyimpan model ke file my_model.h5

In [ ]:
# Fine-tuning
# Buka beberapa lapisan terakhir dari base model
base_model.trainable = True
for layer in base_model.layers[:-20]:  # Bekukan semua kecuali 20 lapisan terakhir
    layer.trainable = False

# Kompilasi ulang model dengan learning rate yang lebih kecil
model.compile(optimizer=Adam(learning_rate=FINE_TUNE_LR),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Fine-tuning model
history_fine_tune = model.fit(
    train_generator,
    epochs=FINE_TUNE_EPOCHS,
    validation_data=test_generator,
    callbacks=[reduce_lr, early_stopping]
)

# Evaluasi model
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")
model.save("my_model.h5")  # Menyimpan model ke file my_model.h5